# Score: 98.971%

I deleted most of Yassine's code and will try to score better by testing things out for myself.

Luckily when I submitted his code I only used 1 epoch so there still is a lot more room to improve

NOTE: I think 15 epochs should be fine next time.  Fitting can take 1.5hrs if going for 30 epochs

In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

train = pd.read_csv("/Users/pbezuhov/git/Kaggle/data/mnist/train.csv")
test = pd.read_csv("/Users/pbezuhov/git/Kaggle/data/mnist/test.csv")

In [2]:
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split

train_y = train['label']
train_x = train.drop(['label'], axis=1)

# Reshape
train_x = train_x.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)

# Normalize
train_x = train_x / 255.0
test = test / 255.0

train_y = to_categorical(train_y, num_classes = 10)

train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size = 0.2)

Using TensorFlow backend.


In [3]:
import tensorflow
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, MaxPool2D, Flatten

model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (5,5), 
                 activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 32, kernel_size = (5,5),
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Conv2D(filters = 64, kernel_size = (3,3),
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),
                 activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

# Compile the model
model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics=["accuracy"])

In [4]:
from keras.preprocessing.image import ImageDataGenerator

data_generator = ImageDataGenerator(
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1  # randomly shift images vertically (fraction of total height)
)


data_generator.fit(train_x)

In [5]:
epochs = 5
batch_size = 86

history = model.fit_generator(data_generator.flow(train_x, train_y, batch_size=batch_size),
                              epochs = epochs, validation_data = (val_x, val_y))

Epoch 1/5
391/391 [==============================] - 221s 564ms/step - loss: 0.6189 - acc: 0.7937 - val_loss: 0.0759 - val_acc: 0.9773
Epoch 2/5
391/391 [==============================] - 176s 449ms/step - loss: 0.1799 - acc: 0.9447 - val_loss: 0.0599 - val_acc: 0.9814
Epoch 3/5
391/391 [==============================] - 162s 415ms/step - loss: 0.1331 - acc: 0.9603 - val_loss: 0.0542 - val_acc: 0.9835
Epoch 4/5
391/391 [==============================] - 188s 482ms/step - loss: 0.1039 - acc: 0.9695 - val_loss: 0.0319 - val_acc: 0.9911
Epoch 5/5
391/391 [==============================] - 196s 500ms/step - loss: 0.0924 - acc: 0.9731 - val_loss: 0.0369 - val_acc: 0.9889


In [6]:
results = model.predict(test)
results = np.argmax(results,axis = 1)
results = pd.Series(results,name="Label")

In [7]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("../submissions/2_redo_redo.csv",index=False)